In [1]:
# %pip install pandas
# %pip install numpy
# %pip install scikit-learn
# %pip install kagglehub
# %pip install cupy

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import kagglehub
import cupy as cp


c:\Users\patry\anaconda3\envs\weather\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
historical_hourly_weather_data_path = kagglehub.dataset_download('selfishgene/historical-hourly-weather-data')

city = "Portland"

city_attributes = pd.read_csv(f"{historical_hourly_weather_data_path}/city_attributes.csv")
humidity = pd.read_csv(f"{historical_hourly_weather_data_path}/humidity.csv")
pressure = pd.read_csv(f"{historical_hourly_weather_data_path}/pressure.csv")
temperature = pd.read_csv(f"{historical_hourly_weather_data_path}/temperature.csv")
weather_description = pd.read_csv(f"{historical_hourly_weather_data_path}/weather_description.csv")
wind_speed = pd.read_csv(f"{historical_hourly_weather_data_path}/wind_speed.csv")
wind_direction = pd.read_csv(f"{historical_hourly_weather_data_path}/wind_direction.csv")

In [4]:
if city not in city_attributes['City'].values:
    raise ValueError(f"City '{city}' does not exist in the data. Available cities are: {city_attributes['City'].unique()}")

selected_city = city_attributes[city_attributes['City'] == city].index[0]
data_frames = [humidity, pressure, temperature, weather_description, wind_speed, wind_direction]

for i, df in enumerate(data_frames):
    df.set_index('datetime', inplace=True)
    data_frames[i] = df.iloc[:, selected_city]

In [5]:
combined_data = pd.concat(data_frames, axis=1)
combined_data.columns = [
    'humidity', 'pressure', 'temperature', 'weather_description', 
    'wind_speed', 'wind_direction'
]
combined_data.index = pd.to_datetime(combined_data.index)
# aggregate daily
aggregated_data = combined_data.resample('D').agg({
    'temperature': 'mean',
    'humidity': 'mean',
    'wind_speed': ['max', 'mean'],
    'pressure': 'mean',
    'weather_description': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'wind_direction': 'mean'
})
aggregated_data.columns = ['_'.join(col).strip('_') for col in aggregated_data.columns.values]

In [6]:
one_hot_weather_description = pd.get_dummies(
    aggregated_data['weather_description_<lambda>'],
    prefix='weather_desc',
    drop_first=False
)

aggregated_data = pd.concat([aggregated_data, one_hot_weather_description], axis=1)
aggregated_data.drop(columns=['weather_description_<lambda>'], inplace=True)
print(aggregated_data.columns)

Index(['temperature_mean', 'humidity_mean', 'wind_speed_max',
       'wind_speed_mean', 'pressure_mean', 'wind_direction_mean',
       'weather_desc_broken clouds', 'weather_desc_dust',
       'weather_desc_few clouds', 'weather_desc_fog',
       'weather_desc_freezing rain', 'weather_desc_haze',
       'weather_desc_heavy intensity rain', 'weather_desc_light rain',
       'weather_desc_mist', 'weather_desc_moderate rain',
       'weather_desc_overcast clouds', 'weather_desc_scattered clouds',
       'weather_desc_sky is clear', 'weather_desc_smoke', 'weather_desc_snow'],
      dtype='object')


In [7]:
binary_output = True

In [8]:
def preprocess_weather_data(data, binary_output=True, window_size=3):
    X, y = [], []
    for i in range(window_size, len(data) - 1):
        X_window = data.iloc[i-window_size:i][[
            'temperature_mean', 'humidity_mean', 'wind_speed_max',
            'wind_speed_mean', 'pressure_mean', 'wind_direction_mean',
            'weather_desc_broken clouds', 'weather_desc_dust',
            'weather_desc_few clouds', 'weather_desc_fog',
            'weather_desc_freezing rain', 'weather_desc_haze',
            'weather_desc_heavy intensity rain', 'weather_desc_light rain',
            'weather_desc_mist', 'weather_desc_moderate rain',
            'weather_desc_overcast clouds', 'weather_desc_scattered clouds',
            'weather_desc_sky is clear', 'weather_desc_smoke', 'weather_desc_snow'
        ]].values
        y_target = data.iloc[i + 1][['temperature_mean', 'wind_speed_max']].values
        # y_target = data.iloc[i][['temperature_mean', 'wind_speed_max']].values

        if binary_output:
            # encode wind speed > 6 as binary
            y_target[1] = 1 if y_target[1] >= 6 else 0
        X.append(X_window)
        y.append(y_target)

    X = np.array(X)
    y = np.array(y)

    train_size = int(0.7 * len(X)) # 0.7 or 0.8
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    continuous_indices = [0, 1, 2, 3, 4, 5]

    X_train_continuous = X_train[:, :, continuous_indices]
    X_test_continuous = X_test[:, :, continuous_indices]
    X_train_continuous = X_train[:, :, continuous_indices].astype(float) 
    X_test_continuous = X_test[:, :, continuous_indices].astype(float)

    X_mean = X_train_continuous.mean(axis=(0, 1), keepdims=True)
    X_std = X_train_continuous.std(axis=(0, 1), keepdims=True)

    X_train[:, :, continuous_indices] = (X_train_continuous - X_mean) / (X_std + 1e-9)
    X_test[:, :, continuous_indices] = (X_test_continuous - X_mean) / (X_std + 1e-9)

    return X_train, X_test, y_train, y_test

In [9]:
X_train, X_test, y_train, y_test = preprocess_weather_data(aggregated_data, binary_output=binary_output, window_size=3)

In [10]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(1318, 3, 21)
(1318, 2)
(565, 3, 21)
(565, 2)


In [11]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=1000, learning_rate=0.001, rate = [500], binary_output=True):
    X_train_cp = cp.array(X_train.reshape(X_train.shape[0], -1), dtype=cp.float32)
    y_train_cp = cp.array(y_train, dtype=cp.float32)
    X_test_cp = cp.array(X_test.reshape(X_test.shape[0], -1), dtype=cp.float32)
    y_test_cp = cp.array(y_test, dtype=cp.float32)

    model.train(X_train_cp, y_train_cp, epochs, learning_rate, rate)

    predictions = model.predict(X_test_cp)
    
    mae = cp.mean(cp.abs(predictions[:, 0] - y_test_cp[:, 0]))

    from sklearn.metrics import roc_auc_score
    if binary_output:
        auc = roc_auc_score(cp.asnumpy(y_test_cp[:, 1]), cp.asnumpy(predictions[:, 1]))
    else:
        auc = roc_auc_score((cp.asnumpy(y_test_cp[:, 1]) >= 6), cp.asnumpy(predictions[:, 1]))

    print(f"Test Regression MAE: {mae}")
    print(f"Test Classification AUC: {auc}")

    return mae, auc

In [12]:
from weather_prediction import WeatherPredictionNetwork

layers = [X_train.shape[1] * X_train.shape[2],512, 512, 2]
activations = ["sigmoid", "relu"]
model = WeatherPredictionNetwork(layers, activations, binary_output=binary_output, seed=42)

train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=10000, learning_rate=0.0001, rate = [2500, 6000, 7500], binary_output=binary_output)

Epoch 0, Regression Loss: 284.46051025390625, Classification AUC: 0.27883763553608826, Learning Rate: 0.0001
Epoch 100, Regression Loss: 250.3007354736328, Classification AUC: 0.6834272004554419, Learning Rate: 0.0001
Epoch 200, Regression Loss: 207.3856201171875, Classification AUC: 0.7015461207560103, Learning Rate: 0.0001
Epoch 300, Regression Loss: 152.6181182861328, Classification AUC: 0.7058507440143371, Learning Rate: 0.0001
Epoch 400, Regression Loss: 85.43536376953125, Classification AUC: 0.7156116660518588, Learning Rate: 0.0001
Epoch 500, Regression Loss: 5.396749019622803, Classification AUC: 0.7102047455421588, Learning Rate: 0.0001
Epoch 600, Regression Loss: 2.341679334640503, Classification AUC: 0.727228630101574, Learning Rate: 0.0001
Epoch 700, Regression Loss: 2.3343052864074707, Classification AUC: 0.732965223175255, Learning Rate: 0.0001
Epoch 800, Regression Loss: 2.3307628631591797, Classification AUC: 0.7361457646515492, Learning Rate: 0.0001
Epoch 900, Regressi

(array(2.39109, dtype=float32), np.float64(0.6226301369863014))

In [13]:
def compute_majority_class_percentage(y, binary_output=True):
    if binary_output:
        binary_labels = y[:, 1]
    else:
        binary_labels = y[:, 1] >=6
    
    unique, counts = np.unique(binary_labels, return_counts=True)
    class_counts = dict(zip(unique, counts))
    
    majority_class_count = max(class_counts.values())
    total_samples = len(binary_labels)
    majority_class_percentage = (majority_class_count / total_samples) * 100
    
    return majority_class_percentage, class_counts

majority_percentage_train, train_class_counts = compute_majority_class_percentage(y_train, binary_output=binary_output)
majority_percentage_test, test_class_counts = compute_majority_class_percentage(y_test, binary_output=binary_output)

print(f"Majority class percentage in training data: {majority_percentage_train:.2f}%")
print(f"Class distribution in training data: {train_class_counts}")
print(f"Majority class percentage in test data: {majority_percentage_test:.2f}%")
print(f"Class distribution in test data: {test_class_counts}")


Majority class percentage in training data: 72.76%
Class distribution in training data: {0: np.int64(959), 1: np.int64(359)}
Majority class percentage in test data: 64.60%
Class distribution in test data: {0: np.int64(365), 1: np.int64(200)}
